In [2]:
pip install -r ../requirements.in

Note: you may need to restart the kernel to use updated packages.


In [3]:
from unstructured.partition.pdf import partition_pdf
from unstructured.documents.elements import Title, NarrativeText, Text
from unstructured.chunking.basic import chunk_elements
from typing import List
import os
import weaviate
from weaviate.util import generate_uuid5
import ollama

In [4]:
ollama.pull("mxbai-embed-large:v1")
ollama.list()

{'models': [{'name': 'gemma:7b',
   'model': 'gemma:7b',
   'modified_at': '2024-04-08T23:55:43.730843947Z',
   'size': 5011853225,
   'digest': 'a72c7f4d0a15522df81486d13ce432c79e191bda2558d024fbad4362c4f7cbf8',
   'details': {'parent_model': '',
    'format': 'gguf',
    'family': 'gemma',
    'families': ['gemma'],
    'parameter_size': '9B',
    'quantization_level': 'Q4_0'}},
  {'name': 'llava:7b',
   'model': 'llava:7b',
   'modified_at': '2024-04-09T00:05:07.649763235Z',
   'size': 4733363377,
   'digest': '8dd30f6b0cb19f555f2c7a7ebda861449ea2cc76bf1f44e262931f45fc81d081',
   'details': {'parent_model': '',
    'format': 'gguf',
    'family': 'llama',
    'families': ['llama', 'clip'],
    'parameter_size': '7B',
    'quantization_level': 'Q4_0'}},
  {'name': 'mistral:7b',
   'model': 'mistral:7b',
   'modified_at': '2024-04-09T00:02:56.351139108Z',
   'size': 4109865159,
   'digest': '61e88e884507ba5e06c49b40e6226884b2a16e872382c2b44a42f2d119d804a5',
   'details': {'parent_mode

In [5]:
FILE_PATH = "../docs/Player_s Handbook.pdf"

elements = partition_pdf(filename=FILE_PATH)

Data-loss while decompressing corrupted data


In [6]:
client = weaviate.connect_to_local()


In [12]:
client.collections.delete(name="PHB")

In [13]:
import weaviate.classes as wvc
collection = client.collections.create(
    name="PHB",
    vectorizer_config=wvc.config.Configure.Vectorizer.none(),
    vector_index_config=wvc.config.Configure.VectorIndex.hnsw(
        distance_metric=wvc.config.VectorDistances.COSINE # select prefered distance metric
    ),
    properties=[
        wvc.config.Property(
            name="Title",
            data_type=wvc.config.DataType.TEXT
        ),
        wvc.config.Property(
            name="NarrativeText",
            data_type=wvc.config.DataType.TEXT
        )
    ]
)

In [19]:
narrative_texts = [elem for elem in elements if elem.category == "NarrativeText"]
titles = [elem for elem in elements if elem.category == "Title"]

text_objects = list()
for index, elem in enumerate(narrative_texts):
    text_objects.append(wvc.data.DataObject(
        properties={
            "NarrativeText": elem["NarrativeText"],
        },
        vector=elem["vector"]
    ))

TypeError: 'NarrativeText' object is not subscriptable